In [2]:
import numpy as np
import perfplot
from numba import njit
from sklearn.datasets import make_regression
import pandas as pd
from scipy.linalg.blas import dgemm
import scipy.linalg as sl
%load_ext snakeviz


ModuleNotFoundError: No module named 'snakeviz'

In [5]:
#2
#generating data for ols
#np.random.seed(25)
variables = 50
nobs = 20000


#def generate_data_ols(nobs, variables):
    
#    means = np.ones(variables)
#    cov = np.eye(variables) + np.ones((variables, variables)) * 0.2
#    x = np.random.multivariate_normal(mean=means, cov=cov, size=nobs)
#    e = np.random.randn(nobs)
#    true_beta = np.random.uniform(-0.99, 0.99, variables)
#    y = x @ true_beta + e

#    return x, y

def generate_data_ols(nobs, variables):
    x, y, coef = make_regression(n_samples=nobs, n_features=variables, n_informative=variables, 
                    effective_rank=None, noise=0.4, shuffle=True, coef=True, random_state=25)
    return x, y, coef

#next steps
#add more implementations using factorisation and spicy
#go on to timing and the other assignment and come back to put everything together
#after all is done fix the other problems



In [6]:
x, y, coef = generate_data_ols(nobs, variables)
x, y, coef

(array([[ 6.53717255e-01, -2.86299195e+00,  4.49643557e-01, ...,
         -8.03092092e-01,  1.81554710e+00,  9.34609956e-01],
        [ 3.96684874e-02, -1.30676025e+00, -2.08085875e+00, ...,
         -5.92928434e-01, -1.04328929e+00,  1.13565534e-01],
        [ 4.43738703e-01, -2.87689834e+00,  2.16870842e-03, ...,
          1.52404394e+00,  4.26721146e-01, -6.37186400e-01],
        ...,
        [ 8.77854741e-02,  7.21123808e-01, -1.84289771e-01, ...,
          1.01052466e+00, -6.55915068e-01,  3.46075638e-02],
        [ 9.76738329e-01,  1.91974900e-01, -7.52095211e-01, ...,
         -1.78340734e+00,  4.41242680e-02, -1.56801849e+00],
        [ 1.23997263e+00,  6.31279143e-01, -2.76662645e-01, ...,
          2.76027227e+00, -1.66727037e+00, -3.12841244e-01]]),
 array([ 4.68705055e+01, -3.06675087e+02, -4.61400131e-01, ...,
         3.90342302e+02,  6.28295735e+02, -5.80269603e+02]),
 array([12.89122851, 99.42609075, 39.69812146, 96.15839456, 13.16011288,
        21.70198703, 69.9007772

In [11]:


"""Define different implementations of OLS using numpy and numpy and spicy.

Each implementation returns the estimated parameter vector.

"""
#---------numpy implementations---------------
#mathematical implemantation numpy
@njit
def matrix_inversion_np(x, y):
    beta = np.linalg.inv(x.T.dot(x)).dot(x.T.dot(y))
    return beta

# least squares implementation numpy
@njit
def lstsq_np(x, y):
    beta = np.linalg.lstsq(x, y)[0]
    return beta

#pseudo inverse implementation numpy
@njit
def pseudo_inverse_np(x, y):
    beta = np.dot(np.linalg.pinv(x), y)
    return beta

# Solve a linear matrix equation, or system of linear scalar equations with numpy.
# this is only valid if x has full rank which is not always the case.
@njit
def solve_np(x,y):
    beta = np.linalg.solve(np.dot(x.T, x), np.dot(x.T, y))
    return beta

def lls_with_blas(x, y, residuals=False):
    """
    https://gist.github.com/aldro61/5889795
    """
    #a, b, coef = generate_data_ols(nobs, variables) - a, b are x, y
    i = dgemm(alpha=1.0, a=a.T, b=a.T, trans_b=True)
    beta = np.linalg.solve(i, dgemm(alpha=1.0, a=a.T, b=b)).flatten()
    return beta

#---------spicy implementations----------------
#matrix multiplication implementation spicy
#@njit
def matrix_inversion_spicy(x, y):
    beta = sl.inv(x.T.dot(x)).dot(x.T.dot(y))
    return beta

# least squares implementation spicy
@njit
def lstsq_spicy(x, y):
    beta = np.linalg.lstsq(x, y)[0]    
    return beta

#pseudo inverse implementation spicy
#@njit
def pseudo_inverse_spicy(x, y):
    beta = np.dot(sl.pinv(x), y)
    return beta

# Solve a linear matrix equation, or system of linear scalar equations with spicy.
# this is only valid if x has full rank which is not always the case.
#@njit
def solve_spicy(x,y):
    beta = sl.solve(np.dot(x.T, x), np.dot(x.T, y))
    return beta



In [12]:
a, b, coef = generate_data_ols(nobs, variables)
a, b, coef

(array([[ 6.53717255e-01, -2.86299195e+00,  4.49643557e-01, ...,
         -8.03092092e-01,  1.81554710e+00,  9.34609956e-01],
        [ 3.96684874e-02, -1.30676025e+00, -2.08085875e+00, ...,
         -5.92928434e-01, -1.04328929e+00,  1.13565534e-01],
        [ 4.43738703e-01, -2.87689834e+00,  2.16870842e-03, ...,
          1.52404394e+00,  4.26721146e-01, -6.37186400e-01],
        ...,
        [ 8.77854741e-02,  7.21123808e-01, -1.84289771e-01, ...,
          1.01052466e+00, -6.55915068e-01,  3.46075638e-02],
        [ 9.76738329e-01,  1.91974900e-01, -7.52095211e-01, ...,
         -1.78340734e+00,  4.41242680e-02, -1.56801849e+00],
        [ 1.23997263e+00,  6.31279143e-01, -2.76662645e-01, ...,
          2.76027227e+00, -1.66727037e+00, -3.12841244e-01]]),
 array([ 4.68705055e+01, -3.06675087e+02, -4.61400131e-01, ...,
         3.90342302e+02,  6.28295735e+02, -5.80269603e+02]),
 array([12.89122851, 99.42609075, 39.69812146, 96.15839456, 13.16011288,
        21.70198703, 69.9007772

In [13]:
#timing the different implementations

time_result = %timeit -o matrix_inversion_np(x,y)
time_result = %timeit -o matrix_inversion_spicy(x,y)
time_result = %timeit -o lstsq_np(x,y)
time_result = %timeit -o lstsq_spicy(x,y)
time_result = %timeit -o pseudo_inverse_np(x,y)
time_result = %timeit -o pseudo_inverse_spicy(x,y)
time_result = %timeit -o solve_np(x,y)
time_result = %timeit -o solve_spicy(x,y)
time_result = %timeit -o lls_with_blas(x,y)

average_time = np.mean(time_result.timings)
average_time

2.23 ms ± 184 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.87 ms ± 303 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
21 ms ± 2.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
23.6 ms ± 2.29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
50.7 ms ± 3.47 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1min 21s ± 2.82 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.3 ms ± 640 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.64 ms ± 220 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.87 ms ± 293 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


0.002873941485714145

SyntaxError: 'return' outside function (<ipython-input-49-c927965af128>, line 24)